# Loading Libraries  

In [70]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import altair as alt
import os.path
import getpass
import platform
from pandasql import sqldf

# Declaring the directory for the script and loading the datasets

In [71]:
username = getpass.getuser()

print("Username: ", username)
pathPart1 = "/Users/"
pathPart2 = "/Github/SCGProject/Datasets/"
complePath = pathPart1+username+pathPart2

if platform.system() == "Darwin" or platform.system() == "Linux":
    if(username == "marcovinciguerra"):
        dfCambio = pd.read_csv(
            complePath+"/CorrectedDatasets/dfCambio.csv")

        dfClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dfClienti.csv")

        dfConsumi = pd.read_csv(
             complePath + "/CorrectedDatasets/dfConsumi.csv")

        dfCostoOrarioConsuntivo = pd.read_csv(
            complePath+"/CorrectedDatasets/dfConsuntivo.csv")

        dfCostoOrarioBudget = pd.read_csv(
            complePath+"CorrectedDatasets/dfCostoOrario.csv")

        dfImpiegoRisorse = pd.read_csv(
         complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

        dftuttiClienti = pd.read_csv(
         complePath+"/CorrectedDatasets/dftuttiClienti.csv")

        dfVendite = pd.read_csv(
         complePath+"/CorrectedDatasets/dfVendite.csv")

    elif(username == "davidguzman"):
            pathPart1 = "/Users/"
            pathPart2 = "/documents/Github/SCGProject/Datasets/"
            complePath = pathPart1+username+pathPart2

            dfCambio = pd.read_csv(
                complePath+"/CorrectedDatasets/dfCambio.csv")

            dfClienti = pd.read_csv(
                complePath+"/CorrectedDatasets/dfClienti.csv")

            dfConsumi = pd.read_csv(
                complePath + "/CorrectedDatasets/dfConsumi.csv")

            dfCostoOrarioConsuntivo = pd.read_csv(
                complePath+"/CorrectedDatasets/dfConsuntivo.csv")

            dfCostoOrarioBudget = pd.read_csv(
                complePath+"CorrectedDatasets/dfCostoOrario.csv")

            dfImpiegoRisorse = pd.read_csv(
            complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

            dftuttiClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dftuttiClienti.csv")

            dfVendite = pd.read_csv(
            complePath+"/CorrectedDatasets/dfVendite.csv")




Username:  marcovinciguerra


# Creating the sales table using joins 

In [72]:
del dfVendite['Unnamed: 0']
dfVendite.head()

,NrMovimento,budget/cons,NrArticolo,NrOrigine,Quantity,ImportoVenditaValutaLocaleTOTALEVENDITA
0,21621,BUDGET,ART0001433,C00140,4,18.80
1,21629,BUDGET,ART0001610,C00140,12,145.28
2,21631,BUDGET,ART0003269,C00140,24,933.99
3,21633,BUDGET,ART0002249,C00140,1,36.33
4,21634,BUDGET,ART0002370,C00140,1,65.72


In [73]:
del dfClienti['Unnamed: 0']
del dfClienti['CodCondizioniPagam']
del dfClienti['FattCumulative']

dfClienti.head()

,Nr,Valuta
0,C00001,1
1,C00003,2
2,C00004,3
3,C00005,1
4,C00006,2


In [74]:
del dfCambio['Unnamed: 0']
dfCambio.head()

,CodiceValuta,Anno,TassoCambioMedio
0,1,BUDGET,1
1,2,BUDGET,"1,0541"
2,3,BUDGET,123.4
3,1,CONSUNTIVO,1
4,2,CONSUNTIVO,"1,1993"


## Join operations for the sales table

In [75]:
temp = sqldf(
    "SELECT  * FROM dfVendite ca JOIN dfClienti cl ON ca.NrOrigine = cl.Nr ")
del temp["Nr"]
temp.columns = ['NrMovimento', 'budget', 'NrArticolo', 'NrOrigine',
                'Quantity', 'ImportoVenditaValutaLocaleTOTALEVENDITA', 'Valuta']

# Renaming the variables 

temp.loc[temp.loc[:, "budget"] == "Consuntivo", "budget"] = "CONSUNTIVO"
temp.loc[temp.loc[:, "budget"] == "Budget", "budget"] = "BUDGET"

tot = sqldf(
    "SELECT * FROM temp t  JOIN dfCambio c ON t.Valuta = c.CodiceValuta AND t.budget = c.Anno ")

del tot["Valuta"]
del tot["Anno"]

tot.head()


,NrMovimento,budget,NrArticolo,NrOrigine,Quantity,ImportoVenditaValutaLocaleTOTALEVENDITA,CodiceValuta,TassoCambioMedio
0,21621,BUDGET,ART0001433,C00140,4,18.80,2,"1,0541"
1,21629,BUDGET,ART0001610,C00140,12,145.28,2,"1,0541"
2,21631,BUDGET,ART0003269,C00140,24,933.99,2,"1,0541"
3,21633,BUDGET,ART0002249,C00140,1,36.33,2,"1,0541"
4,21634,BUDGET,ART0002370,C00140,1,65.72,2,"1,0541"


## Sales table for the final statement

In [76]:
salesFinal = sqldf(
    "SELECT * FROM tot t WHERE t.budget = 'Consuntivo' OR t.budget = 'CONSUNTIVO'")
salesFinal.head()


,NrMovimento,budget,NrArticolo,NrOrigine,Quantity,ImportoVenditaValutaLocaleTOTALEVENDITA,CodiceValuta,TassoCambioMedio
0,35089,CONSUNTIVO,ART0000814,C00140,1,545.20,2,"1,1993"
1,35550,CONSUNTIVO,ART0000841,C00140,1,4650.13,2,"1,1993"
2,35894,CONSUNTIVO,ART0005136,C00483,1,1550.00,2,"1,1993"
3,35944,CONSUNTIVO,ART0000752,C00140,2,912.00,2,"1,1993"
4,35945,CONSUNTIVO,ART0000782,C00140,1,295.00,2,"1,1993"


## Sales table for the budget 

In [77]:
salesBudget = sqldf(
    "SELECT * FROM tot t WHERE t.budget = 'Budget' OR t.budget = 'BUDGET'")
salesBudget.head()


,NrMovimento,budget,NrArticolo,NrOrigine,Quantity,ImportoVenditaValutaLocaleTOTALEVENDITA,CodiceValuta,TassoCambioMedio
0,21621,BUDGET,ART0001433,C00140,4,18.80,2,"1,0541"
1,21629,BUDGET,ART0001610,C00140,12,145.28,2,"1,0541"
2,21631,BUDGET,ART0003269,C00140,24,933.99,2,"1,0541"
3,21633,BUDGET,ART0002249,C00140,1,36.33,2,"1,0541"
4,21634,BUDGET,ART0002370,C00140,1,65.72,2,"1,0541"


# Creating the costs table using joins

In [78]:
del dfConsumi["Unnamed: 0"]

dfConsumi.columns = ['NrMovimento', 'budget', 'CodiceMP', 'NrArticolo',
                     'NrDocumento', 'QuantityMPImpiegata', 'ImportoCostoTOTALE']

dfConsumi.loc[dfConsumi.loc[:, "budget"] == "Consuntivo", "budget"] = "CONSUNTIVO"
dfConsumi.loc[dfConsumi.loc[:, "budget"] == "Budget", "budget"] = "BUDGET"
dfConsumi.head()

,NrMovimento,budget,CodiceMP,NrArticolo,NrDocumento,QuantityMPImpiegata,ImportoCostoTOTALE
0,28854,BUDGET,ART0001074,ART0000018,ODP17-05342,30.0,5.15
1,66819,CONSUNTIVO,ART0001074,ART0000018,ODP19-05998,30.0,5.15
2,28853,BUDGET,ART0005696,ART0000041,ODP17-05342,4.0,92.00
3,66818,CONSUNTIVO,ART0005696,ART0000041,ODP19-05998,4.0,88.00
4,28855,BUDGET,ART0005697,ART0000042,ODP17-05342,4.0,104.00


In [79]:
#Costo orario consuntivo
del dfCostoOrarioConsuntivo["Unnamed: 0"]
dfCostoOrarioConsuntivo


,Risorsa,AreaProduzione,CostoOrario
0,RIS1,A10,60
1,RIS1,A11,60
2,RIS1,A20,80
3,RIS1,A30,80
4,RIS1,A32,60
...,...,...,...
65,RIS9,A11,50
66,RIS9,A20,80
67,RIS9,A30,50
68,RIS9,A32,50


In [80]:
#Budget
del dfCostoOrarioBudget["Unnamed: 0"]
dfCostoOrarioBudget.head()

,Risorsa,AreaProduzione,CostoOrario
0,RIS1,A10,51.0
1,RIS1,A11,51.0
2,RIS1,A20,68.0
3,RIS1,A30,68.0
4,RIS1,A32,51.0


In [81]:
del dfImpiegoRisorse["Unnamed: 0"]
dfImpiegoRisorse.loc[dfImpiegoRisorse.loc[:, "budgetConsuntivo"] == "Consuntivo", "budgetConsuntivo"] = "CONSUNTIVO"
dfImpiegoRisorse.loc[dfImpiegoRisorse.loc[:, "budgetConsuntivo"] == "Budget", "budgetConsuntivo"] = "BUDGET"
dfImpiegoRisorse.head()

,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,QuantitydiOutput
0,ART0000018,BUDGET,ODP17-05342,Tornitura,A20,RIS2,1.00,2.0
1,ART0000018,BUDGET,ODP17-05342,Fresatura,A30,RIS7,0.75,2.0
2,ART0000018,BUDGET,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,2.0
3,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.00,2.0
4,ART0000018,CONSUNTIVO,ODP19-05998,Fresatura,A30,RIS5,1.00,2.0


## Costs table for the final statement

In [82]:
a = sqldf("SELECT * FROM dfImpiegoRisorse WHERE budgetConsuntivo = 'CONSUNTIVO'")

b = sqldf("SELECT * FROM dfConsumi WHERE budget = 'CONSUNTIVO'")

tot = sqldf("SELECT distinct * FROM a AS a join b AS b ON a.NrArticolo = b.NrArticolo and a.NrOrdineProduzione = b.NrDocumento")

totConsuntivo = sqldf("""SELECT distinct NrArticolo, NrOrdineProduzione, Descrizione, NrAreaProduzione, t.Risorsa, TempoRisorsa, CostoOrario as CostoOrarioRisorsa, QuantitydiOutput, 
NrMovimento as NrMovimentoMP, CodiceMP, QuantityMPImpiegata, ImportoCostoTOTALE as ImportoCostoTotaleMP
FROM tot AS t JOIN dfCostoOrarioConsuntivo AS c on t.Risorsa = c.Risorsa and t.NrAreaProduzione = c.AreaProduzione""")
if(username == "davidguzman"):
    totConsuntivo.to_csv("/Users/davidguzman/documents/Github/SCGProject/Datasets/CorrectedDatasets/totConsuntivo.csv")

totConsuntivo.head()


,NrArticolo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrarioRisorsa,QuantitydiOutput,NrMovimentoMP,CodiceMP,QuantityMPImpiegata,ImportoCostoTotaleMP
0,ART0000018,ODP19-05998,Tornitura,A20,RIS2,1.0000,80,2.0,66819,ART0001074,30.0,5.15
1,ART0000018,ODP19-05998,Fresatura,A30,RIS5,1.0000,50,2.0,66819,ART0001074,30.0,5.15
2,ART0000018,ODP19-05998,Controllo qualità,CQ,RIS16,0.0000,15,2.0,66819,ART0001074,30.0,5.15
3,ART0000041,ODP19-05998,Tornitura,A20,RIS1,3.0000,80,4.0,66818,ART0005696,4.0,88.00
4,ART0000041,ODP19-05998,Fresatura,A30,RIS1,0.0001,80,4.0,66818,ART0005696,4.0,88.00


## Costs table for the budget

In [83]:
a = sqldf("SELECT * FROM dfImpiegoRisorse WHERE budgetConsuntivo = 'BUDGET'")

b = sqldf("SELECT * FROM dfConsumi WHERE budget = 'BUDGET'")

tot = sqldf("SELECT distinct * FROM a AS a join b AS b ON a.NrArticolo = b.NrArticolo and a.NrOrdineProduzione = b.NrDocumento")

totBudget = sqldf(
    """SELECT distinct NrArticolo, NrOrdineProduzione, Descrizione, NrAreaProduzione, t.Risorsa, TempoRisorsa, CostoOrario as CostoOrarioRisorsa, QuantitydiOutput, 
NrMovimento as NrMovimentoMP, CodiceMP, QuantityMPImpiegata, ImportoCostoTOTALE as ImportoCostoTotaleMP 
    FROM tot AS t JOIN  dfCostoOrarioBudget AS c ON t.Risorsa = c.Risorsa and t.NrAreaProduzione = c.AreaProduzione""")
if(username == "davidguzman"):
    totBudget.to_csv("/Users/davidguzman/documents/Github/SCGProject/Datasets/CorrectedDatasets/totBudget.csv")

totBudget.head()


,NrArticolo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrarioRisorsa,QuantitydiOutput,NrMovimentoMP,CodiceMP,QuantityMPImpiegata,ImportoCostoTotaleMP
0,ART0000018,ODP17-05342,Tornitura,A20,RIS2,1.00,68.00,2.0,28854,ART0001074,30.0,5.15
1,ART0000018,ODP17-05342,Fresatura,A30,RIS7,0.75,51.00,2.0,28854,ART0001074,30.0,5.15
2,ART0000018,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,12.75,2.0,28854,ART0001074,30.0,5.15
3,ART0000041,ODP17-05342,Tornitura,A20,RIS14,3.00,51.00,4.0,28853,ART0005696,4.0,92.00
4,ART0000041,ODP17-05342,Tornitura,A20,RIS14,1.50,51.00,0.0,28853,ART0005696,4.0,92.00


# 1) Scostamento costo totale Materia Prima

In [84]:
consumiConsuntivo = sqldf("SELECT DISTINCT * FROM dfConsumi WHERE budget = 'CONSUNTIVO'")
consumiBudget = sqldf("SELECT DISTINCT * FROM dfConsumi WHERE budget = 'BUDGET'")
sumConsumiConsuntivo = sqldf('select sum(ImportoCostoTOTALE) from consumiConsuntivo')
sumConsumiBudget = sqldf('select sum(ImportoCostoTOTALE) from consumiBudget')
sumConsumiBudget-sumConsumiConsuntivo

,sum(ImportoCostoTOTALE)
0,-96733.03


# 1.1) Scostamento costo totale per CodiceMP

## group by CodiceMP

In [85]:
# quantità totale di codici MP 
countConsumiMPc = sqldf('select distinct CodiceMP, count(*) from consumiConsuntivo group by CodiceMP order by count(*) desc')
countConsumiMPb = sqldf('select distinct CodiceMP, count(*) from consumiBudget group by CodiceMP order by count(*) desc')
print("Consuntivo")
print(countConsumiMPc)
print("Budget")
print(countConsumiMPb)
#Considerazione= Consuntivo ha più codici rispetto a budget
# 629-553 = 76 codici single

Consuntivo
       CodiceMP  count(*)
0    ART0005176       116
1    ART0005216        64
2    ART0003831        64
3    ART0005139        58
4    ART0001387        49
..          ...       ...
625  ART0001075         1
626  ART0001068         1
627  ART0001066         1
628  ART0000809         1
629  ART0000771         1

[630 rows x 2 columns]
Budget
       CodiceMP  count(*)
0    ART0003832        33
1    ART0003831        32
2    ART0005160        26
3    ART0005139        26
4    ART0005227        23
..          ...       ...
549  ART0001080         1
550  ART0001078         1
551  ART0001077         1
552  ART0001076         1
553  ART0001073         1

[554 rows x 2 columns]


In [86]:
consumiConsuntivoByMP = sqldf('select codiceMP, sum(importoCostoTOTALE) as sumCostoMP from consumiConsuntivo group by codiceMP order by sumCostoMP desc')
consumiBudgetByMP = sqldf('select codiceMP, sum(importoCostoTOTALE) as sumCostoMP from consumiBudget group by codiceMP order by sumCostoMP desc')

In [87]:
#Join e differenza tra le 2 table precedenti
scostamentoConsumiMP = sqldf('''select c.CodiceMP, b.sumCostoMP as sumCostoMPBudget, c.sumCostoMP as sumCostoMPConsuntivo, b.sumCostoMP-c.sumCostoMP as scostamento 
from consumiConsuntivoByMP as c join consumiBudgetByMP as b 
on c.codiceMP = b.codiceMP
order by scostamento desc''')

scostamentoConsumiMP

,CodiceMP,sumCostoMPBudget,sumCostoMPConsuntivo,scostamento
0,ART0001897,10186.10,2038.64,8147.46
1,ART0001785,4232.98,936.17,3296.81
2,ART0001931,2428.44,290.83,2137.61
3,ART0001942,2622.83,709.97,1912.86
4,ART0005160,5098.10,3524.94,1573.16
...,...,...,...,...
466,ART0001940,2352.81,6347.85,-3995.04
467,ART0001770,240.59,5261.44,-5020.85
468,ART0001756,4362.04,10546.20,-6184.16
469,ART0001883,400.37,8330.78,-7930.41


In [88]:
sqldf('select sum (scostamento) from scostamentoConsumiMP') #Considero solo quelli che sono presenti in entrambi
# notiamo che lo scostamento totale continua essere in linea con quello orginale, 
# la differenza potrebbe essere giustificata da MP senza con budget e senza consuntivo o viceversa

,sum (scostamento)
0,-90466.33


### Versione con full join (not supported by SQL)

In [89]:
#scostamentoConsumiMP = sqldf('''select c.CodiceMP, b.sumCostoMP as sumCostoMPBudget, c.sumCostoMP as sumCostoMPConsuntivo, b.sumCostoMP-c.sumCostoMP as scostamento 
#from consumiConsuntivoByMP as c full join consumiBudgetByMP as b 
#on c.codiceMP = b.codiceMP
#order by scostamento desc''')

#scostamentoConsumiMP

## scostamento di impiego e prezzo

In [90]:
consumiConsuntivoUnit = sqldf('''select distinct NrMovimento, CodiceMP, NrArticolo, QuantityMPImpiegata, ImportoCostoTOTALE/QuantityMPImpiegata as CostoUnitario, ImportoCostoTOTALE 
from consumiConsuntivo
order by CodiceMP''')
consumiBudgetUnit = sqldf('''select distinct NrMovimento, CodiceMP, NrArticolo, QuantityMPImpiegata, ImportoCostoTOTALE/QuantityMPImpiegata as CostoUnitario, ImportoCostoTOTALE 
from consumiBudget
order by CodiceMP''')

In [91]:
consumiConsuntivoUnitByMP = sqldf('''select distinct CodiceMP, sum(QuantityMPImpiegata) as sumQuantityMPImpiegata, sum(ImportoCostoTOTALE)/sum(QuantityMPImpiegata) as CostoUnitario, sum(ImportoCostoTOTALE) as sumCostoMP
from consumiConsuntivo
group by CodiceMP
order by CodiceMP''')
consumiBudgetUnitByMP = sqldf('''select distinct CodiceMP, sum(QuantityMPImpiegata) as sumQuantityMPImpiegata, sum(ImportoCostoTOTALE)/sum(QuantityMPImpiegata) as CostoUnitario, sum(ImportoCostoTOTALE) as sumCostoMP
from consumiBudget
group by CodiceMP
order by CodiceMP''')

### scostamento di impiego

In [92]:
scostamentoConsumiMPImpiego = sqldf('''select c.CodiceMP, b.sumCostoMP as ImpiegoStandard, c.sumQuantityMPImpiegata*b.CostoUnitario as ImpiegoEffettivo, b.sumCostoMP-c.sumQuantityMPImpiegata*b.CostoUnitario as scostamento
from consumiConsuntivoUnitByMP as c join consumiBudgetUnitByMP as b 
on c.CodiceMP = b.CodiceMP
order by c.CodiceMP''')
scostamentoConsumiMPImpiego

,CodiceMP,ImpiegoStandard,ImpiegoEffettivo,scostamento
0,ART0001073,2.59,25.554667,-22.964667
1,ART0001074,203.48,253.791932,-50.311932
2,ART0001076,158.70,844.905000,-686.205000
3,ART0001078,5.47,47.295741,-41.825741
4,ART0001080,1.21,2.634677,-1.424677
...,...,...,...,...
466,ART0005934,25.83,599.966667,-574.136667
467,ART0005944,0.00,0.000000,0.000000
468,ART0005951,148.05,1035.504000,-887.454000
469,ART0005953,826.57,28.502414,798.067586


In [93]:
tempImpiego = sqldf('select sum(scostamento) from scostamentoConsumiMPImpiego')
tempImpiego

,sum(scostamento)
0,-91732.950316


### scostamento di prezzo

In [94]:
scostamentoConsumiMPPrezzo = sqldf('''select c.CodiceMP, c.sumQuantityMPImpiegata*b.CostoUnitario as ImpiegoEffettivo, c.sumCostoMP as BudgetEffettivo, c.sumQuantityMPImpiegata*b.CostoUnitario-c.sumCostoMP as scostamento
from consumiConsuntivoUnitByMP as c join consumiBudgetUnitByMP as b 
on c.CodiceMP = b.CodiceMP
order by c.CodiceMP''')
scostamentoConsumiMPPrezzo 

,CodiceMP,ImpiegoEffettivo,BudgetEffettivo,scostamento
0,ART0001073,25.554667,25.01,0.544667
1,ART0001074,253.791932,257.05,-3.258068
2,ART0001076,844.905000,890.51,-45.605000
3,ART0001078,47.295741,67.26,-19.964259
4,ART0001080,2.634677,2.97,-0.335323
...,...,...,...,...
466,ART0005934,599.966667,649.49,-49.523333
467,ART0005944,0.000000,0.00,0.000000
468,ART0005951,1035.504000,1004.04,31.464000
469,ART0005953,28.502414,42.40,-13.897586


In [95]:
tempPrezzo = sqldf('select sum(scostamento) from scostamentoConsumiMPPrezzo')
tempPrezzo

,sum(scostamento)
0,1266.620316


In [96]:
tempImpiego+tempPrezzo # scostamento totale resta uguale, TODO full join

,sum(scostamento)
0,-90466.33


# 1.2) Scostamento costo totale per articolo

In [97]:
# quantità totale di articoli
countConsumiMPc = sqldf('select distinct NrArticolo, count(*) from consumiConsuntivo group by NrArticolo order by count(*) desc')
countConsumiMPb = sqldf('select distinct NrArticolo, count(*) from consumiBudget group by NrArticolo order by count(*) desc')
print(countConsumiMPc)
print(countConsumiMPb) #Stesso numero di articoli

     NrArticolo  count(*)
0    ART0005136       234
1    ART0000782       184
2    ART0000752       133
3    ART0000847       128
4    ART0000748       119
..          ...       ...
499  ART0000115         1
500  ART0000043         1
501  ART0000042         1
502  ART0000041         1
503  ART0000018         1

[504 rows x 2 columns]
     NrArticolo  count(*)
0    ART0000795        66
1    ART0000762        66
2    ART0000782        64
3    ART0004099        52
4    ART0000752        49
..          ...       ...
499  ART0000119         1
500  ART0000043         1
501  ART0000042         1
502  ART0000041         1
503  ART0000018         1

[504 rows x 2 columns]


In [98]:
consumiConsuntivoByArt = sqldf('select NrArticolo, sum(importoCostoTOTALE) as sumCostoMP from consumiConsuntivo group by NrArticolo order by sumCostoMP desc')
consumiBudgetByArt = sqldf('select NrArticolo, sum(importoCostoTOTALE) as sumCostoMP from consumiBudget group by NrArticolo order by sumCostoMP desc')

In [99]:
scostamentoConsumiArt = sqldf('''select c.NrArticolo, b.sumCostoMP as sumCostoMPBudget, c.sumCostoMP as sumCostoMPConsuntivo, b.sumCostoMP-c.sumCostoMP as scostamento 
from consumiConsuntivoByArt as c join consumiBudgetByArt as b 
on c.NrArticolo = b.NrArticolo
order by scostamento desc''')

scostamentoConsumiArt

,NrArticolo,sumCostoMPBudget,sumCostoMPConsuntivo,scostamento
0,ART0000762,15123.70,3111.43,12012.27
1,ART0000826,3913.21,651.27,3261.94
2,ART0000812,3900.33,1246.50,2653.83
3,ART0004146,1790.92,605.97,1184.95
4,ART0001913,1469.78,524.13,945.65
...,...,...,...,...
499,ART0000752,6489.77,12505.88,-6016.11
500,ART0000811,3343.14,10540.89,-7197.75
501,ART0000853,11467.17,18994.86,-7527.69
502,ART0000847,706.68,15023.61,-14316.93


In [100]:
sqldf('select sum (scostamento) from scostamentoConsumiArt') 
# notiamo che lo scostamento totale è completamente in linea con quello orginale 

,sum (scostamento)
0,-96733.03


## scostamento di impiego e prezzo

In [101]:
consumiConsuntivoUnit = sqldf('''select distinct NrMovimento, CodiceMP, NrArticolo, QuantityMPImpiegata, ImportoCostoTOTALE/QuantityMPImpiegata as CostoUnitario, ImportoCostoTOTALE 
from consumiConsuntivo
order by CodiceMP''')
consumiBudgetUnit = sqldf('''select distinct NrMovimento, CodiceMP, NrArticolo, QuantityMPImpiegata, ImportoCostoTOTALE/QuantityMPImpiegata as CostoUnitario, ImportoCostoTOTALE 
from consumiBudget
order by CodiceMP''')

In [102]:
# sqldf('select NrArticolo,sum(QuantityMPImpiegata) as sumQuantityMPImpiegata, sum(CostoUnitario) as sumCostoUnitario, sum(ImportoCostoTOTALE) as sumCostoMP from consumiConsuntivoUnit group by NrArticolo')
consumiConsuntivoUnitByArt = sqldf('''select distinct NrArticolo, sum(QuantityMPImpiegata) as sumQuantityMPImpiegata, sum(ImportoCostoTOTALE)/sum(QuantityMPImpiegata) as CostoUnitario, sum(ImportoCostoTOTALE) as sumCostoMP
from consumiConsuntivo
group by NrArticolo
order by NrArticolo''')
consumiBudgetUnitByArt = sqldf('''select distinct NrArticolo, sum(QuantityMPImpiegata) as sumQuantityMPImpiegata, sum(ImportoCostoTOTALE)/sum(QuantityMPImpiegata) as CostoUnitario, sum(ImportoCostoTOTALE) as sumCostoMP
from consumiBudget
group by NrArticolo
order by NrArticolo''')

### scostamento di impiego

In [103]:
scostamentoConsumiArtImpiego = sqldf('''select c.NrArticolo, b.sumCostoMP as ImpiegoStandard, c.sumQuantityMPImpiegata*b.CostoUnitario as ImpiegoEffettivo, b.sumCostoMP-c.sumQuantityMPImpiegata*b.CostoUnitario as scostamento
from consumiConsuntivoUnitByArt as c join consumiBudgetUnitByArt as b 
on c.NrArticolo = b.NrArticolo
order by c.NrArticolo''')
scostamentoConsumiArtImpiego


,NrArticolo,ImpiegoStandard,ImpiegoEffettivo,scostamento
0,ART0000018,5.15,5.150000,0.000000
1,ART0000041,92.00,92.000000,0.000000
2,ART0000042,104.00,104.000000,0.000000
3,ART0000043,36.06,35.716571,0.343429
4,ART0000115,434.44,216.720644,217.719356
...,...,...,...,...
499,ART0005109,6.97,15.891600,-8.921600
500,ART0005110,74.00,407.000000,-333.000000
501,ART0005111,16.00,64.000000,-48.000000
502,ART0005112,26.00,104.000000,-78.000000


In [104]:
tempImpiego = sqldf('select sum(scostamento) from scostamentoConsumiArtImpiego')
tempImpiego

,sum(scostamento)
0,-337819.267312


### scostamento di prezzo

In [105]:
scostamentoConsumiArtPrezzo = sqldf('''select c.NrArticolo, c.sumQuantityMPImpiegata*b.CostoUnitario as ImpiegoEffettivo, c.sumCostoMP as BudgetEffettivo, c.sumQuantityMPImpiegata*b.CostoUnitario-c.sumCostoMP as scostamento
from consumiConsuntivoUnitByArt as c join consumiBudgetUnitByArt as b 
on c.NrArticolo = b.NrArticolo
order by c.NrArticolo''')
scostamentoConsumiArtPrezzo 

,NrArticolo,ImpiegoEffettivo,BudgetEffettivo,scostamento
0,ART0000018,5.150000,5.15,0.000000
1,ART0000041,92.000000,88.00,4.000000
2,ART0000042,104.000000,100.00,4.000000
3,ART0000043,35.716571,35.71,0.006571
4,ART0000115,216.720644,268.26,-51.539356
...,...,...,...,...
499,ART0005109,15.891600,14.74,1.151600
500,ART0005110,407.000000,304.00,103.000000
501,ART0005111,64.000000,64.00,0.000000
502,ART0005112,104.000000,104.00,0.000000


In [106]:
tempPrezzo = sqldf('select sum(scostamento) from scostamentoConsumiArtPrezzo')
tempPrezzo

,sum(scostamento)
0,241086.237312


In [107]:
tempImpiego+tempPrezzo # scostamento totale remains consistent!!! :) 

,sum(scostamento)
0,-96733.03


# 2) Scostamenti lavorazioni interne

In [108]:
dfImpiegoRisorse

,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,QuantitydiOutput
0,ART0000018,BUDGET,ODP17-05342,Tornitura,A20,RIS2,1.00,2.0
1,ART0000018,BUDGET,ODP17-05342,Fresatura,A30,RIS7,0.75,2.0
2,ART0000018,BUDGET,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,2.0
3,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.00,2.0
4,ART0000018,CONSUNTIVO,ODP19-05998,Fresatura,A30,RIS5,1.00,2.0
...,...,...,...,...,...,...,...,...
6259,ART0005136,CONSUNTIVO,ODP19-02674,Fresatura,A30,RIS10,2.25,31.0
6260,ART0005136,CONSUNTIVO,ODP19-02674,Controllo qualità,CQ,RIS16,0.00,40.0
6261,ART0005136,CONSUNTIVO,ODP19-06688,Tornitura,A20,RIS2,1.50,9.0
6262,ART0005136,CONSUNTIVO,ODP19-06688,Tornitura,A20,RIS2,-0.50,0.0


In [109]:
impiegoConsuntivo = sqldf("SELECT DISTINCT * FROM dfImpiegoRisorse WHERE budgetConsuntivo = 'CONSUNTIVO' or budgetConsuntivo = 'Consuntivo'")
impiegoConsuntivo  = sqldf('''SELECT DISTINCT i.NrArticolo,i.budgetConsuntivo,i.NrOrdineProduzione,i.Descrizione,i.NrAreaProduzione,i.Risorsa,i.QuantitydiOutput,i.TempoRisorsa,c.CostoOrario
FROM impiegoConsuntivo as i join dfCostoOrarioConsuntivo as c 
on c.AreaProduzione = i.NrAreaProduzione and c.Risorsa = i.Risorsa''')
impiegoConsuntivo 

,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,QuantitydiOutput,TempoRisorsa,CostoOrario
0,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,2.0,1.0000,80
1,ART0000018,CONSUNTIVO,ODP19-05998,Fresatura,A30,RIS5,2.0,1.0000,50
2,ART0000018,CONSUNTIVO,ODP19-05998,Controllo qualità,CQ,RIS16,2.0,0.0000,15
3,ART0000041,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS1,4.0,3.0000,80
4,ART0000041,CONSUNTIVO,ODP19-05998,Fresatura,A30,RIS1,4.0,0.0001,80
...,...,...,...,...,...,...,...,...,...
3718,ART0005136,CONSUNTIVO,ODP19-02674,Fresatura,A30,RIS10,31.0,2.2500,60
3719,ART0005136,CONSUNTIVO,ODP19-02674,Controllo qualità,CQ,RIS16,40.0,0.0000,15
3720,ART0005136,CONSUNTIVO,ODP19-06688,Tornitura,A20,RIS2,9.0,1.5000,80
3721,ART0005136,CONSUNTIVO,ODP19-06688,Tornitura,A20,RIS2,0.0,-0.5000,80


risolvere problema di quantità di output nulle:

In [110]:
sqldf('''select NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrario, count(QuantitydiOutput)
from impiegoConsuntivo
group by NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrario
order by count(QuantitydiOutput) desc
''') # dunque bisogna sommare le quantità di output a parità di tutte le altre colonne

,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrario,count(QuantitydiOutput)
0,ART0005136,CONSUNTIVO,ODP18-02453,Controllo qualità,CQ,RIS16,0.0,15,3
1,ART0005136,CONSUNTIVO,ODP19-01877,Controllo qualità,CQ,RIS16,0.0,15,3
2,ART0000115,CONSUNTIVO,ODP19-00633,Tornitura,A20,RIS1,9.0,80,2
3,ART0000129,CONSUNTIVO,ODP18-05141,Tornitura,A20,RIS1,3.0,80,2
4,ART0000226,CONSUNTIVO,ODP18-02445,Tornitura,A20,RIS2,3.5,80,2
...,...,...,...,...,...,...,...,...,...
3653,ART0005136,CONSUNTIVO,ODP19-06533,Controllo qualità,CQ,RIS16,0.0,15,1
3654,ART0005136,CONSUNTIVO,ODP19-06533,Tornitura,A20,RIS13,0.5,80,1
3655,ART0005136,CONSUNTIVO,ODP19-06688,Controllo qualità,CQ,RIS16,0.0,15,1
3656,ART0005136,CONSUNTIVO,ODP19-06688,Tornitura,A20,RIS2,-0.5,80,1


In [111]:
impiegoConsuntivo = sqldf('''select NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,sum(TempoRisorsa) as TempoRisorsa,CostoOrario, sum(QuantitydiOutput) as QuantitydiOutput
from impiegoConsuntivo
group by NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrario
order by NrArticolo
''')
impiegoConsuntivo

,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrario,QuantitydiOutput
0,ART0000018,CONSUNTIVO,ODP19-05998,Controllo qualità,CQ,RIS16,0.0000,15,2.0
1,ART0000018,CONSUNTIVO,ODP19-05998,Fresatura,A30,RIS5,1.0000,50,2.0
2,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.0000,80,2.0
3,ART0000041,CONSUNTIVO,ODP19-05998,Controllo qualità,CQ,RIS16,0.0000,15,4.0
4,ART0000041,CONSUNTIVO,ODP19-05998,Fresatura,A30,RIS1,0.0001,80,4.0
...,...,...,...,...,...,...,...,...,...
3012,ART0005136,CONSUNTIVO,ODP19-06532,Fresatura,A30,RIS14,1.2500,50,3.0
3013,ART0005136,CONSUNTIVO,ODP19-06533,Controllo qualità,CQ,RIS16,0.0000,15,4.0
3014,ART0005136,CONSUNTIVO,ODP19-06533,Tornitura,A20,RIS13,0.5000,80,4.0
3015,ART0005136,CONSUNTIVO,ODP19-06688,Controllo qualità,CQ,RIS16,0.0000,15,9.0


Impiego budget

In [112]:
impiegoBudget = sqldf("SELECT DISTINCT * FROM dfImpiegoRisorse WHERE budgetConsuntivo = 'BUDGET' or budgetConsuntivo = 'Budget'")
impiegoBudget  = sqldf('''SELECT DISTINCT i.NrArticolo,i.budgetConsuntivo,i.NrOrdineProduzione,i.Descrizione,i.NrAreaProduzione,i.Risorsa,i.QuantitydiOutput,i.TempoRisorsa,c.CostoOrario
FROM impiegoBudget as i join dfCostoOrarioConsuntivo as c 
on c.AreaProduzione = i.NrAreaProduzione and c.Risorsa = i.Risorsa''')
impiegoBudget # righe a consuntivo sono di più che a budget e la somma dei 2 non corrisponde al numero di righe della tabella impiego risorse

,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,QuantitydiOutput,TempoRisorsa,CostoOrario
0,ART0000018,BUDGET,ODP17-05342,Tornitura,A20,RIS2,2.0,1.00,80
1,ART0000018,BUDGET,ODP17-05342,Fresatura,A30,RIS7,2.0,0.75,60
2,ART0000018,BUDGET,ODP17-05342,Controllo qualità,CQ,RIS16,2.0,0.00,15
3,ART0000041,BUDGET,ODP17-05342,Tornitura,A20,RIS14,4.0,3.00,60
4,ART0000041,BUDGET,ODP17-05342,Tornitura,A20,RIS14,0.0,1.50,60
...,...,...,...,...,...,...,...,...,...
2445,ART0005136,BUDGET,ODP17-15077,Tornitura,A20,RIS14,10.0,1.25,60
2446,ART0005136,BUDGET,ODP17-15077,Controllo qualità,CQ,RIS16,10.0,0.00,15
2447,ART0005136,BUDGET,ODP17-16230,Tornitura,A20,RIS13,1.0,0.75,80
2448,ART0005136,BUDGET,ODP17-16230,Brocciatura / stozzatrice,A33,RIS13,1.0,0.25,50


risolvere problema di quantità di output nulle:

In [113]:
sqldf('''select NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrario, count(QuantitydiOutput)
from impiegoBudget
group by NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrario
order by count(QuantitydiOutput) desc
''') 

,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrario,count(QuantitydiOutput)
0,ART0000116,BUDGET,ODP17-02128,Fresatura,A30,RIS9,1.50,50,2
1,ART0000320,BUDGET,ODP17-10637,Fresatura,A30,RIS7,0.50,60,2
2,ART0000628,BUDGET,ODP17-02528,Fresatura,A30,RIS9,9.00,50,2
3,ART0000630,BUDGET,ODP17-09327,Fresatura,A30,RIS9,4.00,50,2
4,ART0000644,BUDGET,ODP17-02392,Fresatura,A30,RIS9,6.75,50,2
...,...,...,...,...,...,...,...,...,...
2398,ART0005136,BUDGET,ODP17-15077,Controllo qualità,CQ,RIS16,0.00,15,1
2399,ART0005136,BUDGET,ODP17-15077,Tornitura,A20,RIS14,1.25,60,1
2400,ART0005136,BUDGET,ODP17-16230,Brocciatura / stozzatrice,A33,RIS13,0.25,50,1
2401,ART0005136,BUDGET,ODP17-16230,Controllo qualità,CQ,RIS16,0.00,15,1


In [114]:
impiegoBudget = sqldf('''select NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,sum(TempoRisorsa) as TempoRisorsa,CostoOrario, sum(QuantitydiOutput) as QuantitydiOutput
from impiegoBudget
group by NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrario
order by NrArticolo
''')
impiegoBudget

,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrario,QuantitydiOutput
0,ART0000018,BUDGET,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,15,2.0
1,ART0000018,BUDGET,ODP17-05342,Fresatura,A30,RIS7,0.75,60,2.0
2,ART0000018,BUDGET,ODP17-05342,Tornitura,A20,RIS2,1.00,80,2.0
3,ART0000041,BUDGET,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,15,4.0
4,ART0000041,BUDGET,ODP17-05342,Fresatura,A30,RIS7,1.25,60,4.0
...,...,...,...,...,...,...,...,...,...
2006,ART0005136,BUDGET,ODP17-15077,Controllo qualità,CQ,RIS16,0.00,15,10.0
2007,ART0005136,BUDGET,ODP17-15077,Tornitura,A20,RIS14,1.25,60,10.0
2008,ART0005136,BUDGET,ODP17-16230,Brocciatura / stozzatrice,A33,RIS13,0.25,50,1.0
2009,ART0005136,BUDGET,ODP17-16230,Controllo qualità,CQ,RIS16,0.00,15,1.0


In [115]:
a = sqldf('select NrArticolo,Risorsa,NrAreaProduzione,Descrizione, count(*) as count from impiegoConsuntivo group by NrArticolo,Risorsa,NrAreaProduzione,Descrizione,NrOrdineProduzione order by count(*) desc')
b = sqldf('select NrArticolo,Risorsa,NrAreaProduzione,Descrizione, count(*) as count from impiegoBudget group by NrArticolo,Risorsa,NrAreaProduzione,Descrizione,NrOrdineProduzione order by count(*) desc')
c = sqldf('select a.NrArticolo, a.count as consuntivo, b.count as budget from a join b on a.NrArticolo = b.NrArticolo and a.Risorsa = b.Risorsa and a.NrAreaProduzione = b.NrAreaProduzione and a.Descrizione = b.Descrizione')
cproblem = sqldf('select * from c where consuntivo != budget')
c

,NrArticolo,consuntivo,budget
0,ART0000018,1,1
1,ART0000018,1,1
2,ART0000041,1,1
3,ART0000042,1,1
4,ART0000043,1,1
...,...,...,...
5290,ART0005136,1,1
5291,ART0005136,1,1
5292,ART0005136,1,1
5293,ART0005136,1,1


scostamento totale

In [116]:
sumImpiegoConsuntivo = sqldf('SELECT sum(QuantitydiOutput*TempoRisorsa*CostoOrario) FROM impiegoConsuntivo')
sumImpiegoBudget = sqldf('SELECT sum(QuantitydiOutput*TempoRisorsa*CostoOrario) FROM impiegoBudget')
print(sumImpiegoBudget)
print(sumImpiegoConsuntivo)
print(sumImpiegoBudget-sumImpiegoConsuntivo)

   sum(QuantitydiOutput*TempoRisorsa*CostoOrario)
0                                       2802273.7
   sum(QuantitydiOutput*TempoRisorsa*CostoOrario)
0                                    5.349020e+06
   sum(QuantitydiOutput*TempoRisorsa*CostoOrario)
0                                   -2.546746e+06


## 2.1) Scostamento di volume, impiego, prezzo

In [117]:
sqldf(''' select distinct *
FROM impiegoConsuntivo as c join impiegoBudget as b 
on c.NrArticolo=b.NrArticolo and c.Risorsa=b.Risorsa and c.NrAreaProduzione=b.NrAreaProduzione and c.Descrizione=b.Descrizione ''') 

,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrario,QuantitydiOutput,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrario,QuantitydiOutput
0,ART0000018,CONSUNTIVO,ODP19-05998,Controllo qualità,CQ,RIS16,0.00,15,2.0,ART0000018,BUDGET,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,15,2.0
1,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.00,80,2.0,ART0000018,BUDGET,ODP17-05342,Tornitura,A20,RIS2,1.00,80,2.0
2,ART0000041,CONSUNTIVO,ODP19-05998,Controllo qualità,CQ,RIS16,0.00,15,4.0,ART0000041,BUDGET,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,15,4.0
3,ART0000042,CONSUNTIVO,ODP19-05998,Controllo qualità,CQ,RIS16,0.00,15,4.0,ART0000042,BUDGET,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,15,4.0
4,ART0000043,CONSUNTIVO,ODP19-05998,Brocciatura / stozzatrice,A33,RIS13,0.75,50,2.0,ART0000043,BUDGET,ODP17-05342,Brocciatura / stozzatrice,A33,RIS13,0.50,50,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5290,ART0005136,CONSUNTIVO,ODP19-06688,Tornitura,A20,RIS2,1.00,80,9.0,ART0005136,BUDGET,ODP17-10340,Tornitura,A20,RIS2,0.50,80,2.0
5291,ART0005136,CONSUNTIVO,ODP19-06688,Tornitura,A20,RIS2,1.00,80,9.0,ART0005136,BUDGET,ODP17-10463,Tornitura,A20,RIS2,2.25,80,5.0
5292,ART0005136,CONSUNTIVO,ODP19-06688,Tornitura,A20,RIS2,1.00,80,9.0,ART0005136,BUDGET,ODP17-11970,Tornitura,A20,RIS2,2.25,80,56.0
5293,ART0005136,CONSUNTIVO,ODP19-06688,Tornitura,A20,RIS2,1.00,80,9.0,ART0005136,BUDGET,ODP17-11971,Tornitura,A20,RIS2,6.00,80,4.0


In [118]:
scostamentoImpiegoVolume = sqldf(
''' SELECT sum(b.QuantitydiOutput*b.TempoRisorsa*b.CostoOrario) as BudgetStandard, sum(c.QuantitydiOutput*b.TempoRisorsa*b.CostoOrario) as ImpiegoStandard, sum(b.QuantitydiOutput*b.TempoRisorsa*b.CostoOrario)-sum(c.QuantitydiOutput*b.TempoRisorsa*b.CostoOrario) as scostamento
FROM impiegoConsuntivo as c join impiegoBudget as b 
on c.NrArticolo=b.NrArticolo and c.Risorsa=b.Risorsa and c.NrAreaProduzione=b.NrAreaProduzione
''')

scostamentoImpiegoVolume

,BudgetStandard,ImpiegoStandard,scostamento
0,6300260.412,5.016818e+06,1.283443e+06


In [119]:
scostamentoImpiegoImpiego = sqldf(
''' SELECT  sum(c.QuantitydiOutput*b.TempoRisorsa*b.CostoOrario) as ImpiegoStandard,sum(c.QuantitydiOutput*c.TempoRisorsa*b.CostoOrario) as ImpiegoEffettivo, sum(c.QuantitydiOutput*b.TempoRisorsa*b.CostoOrario)-sum(c.QuantitydiOutput*c.TempoRisorsa*b.CostoOrario) as scostamento
FROM impiegoConsuntivo as c join impiegoBudget as b 
on c.NrArticolo=b.NrArticolo and c.Risorsa=b.Risorsa and c.NrAreaProduzione=b.NrAreaProduzione
''')

scostamentoImpiegoImpiego

,ImpiegoStandard,ImpiegoEffettivo,scostamento
0,5.016818e+06,5.341534e+06,-324716.0676


In [120]:
scostamentoImpiegoPrezzo = sqldf(
''' SELECT  sum(c.QuantitydiOutput*c.TempoRisorsa*b.CostoOrario) as ImpiegoEffettivo, sum(c.QuantitydiOutput*c.TempoRisorsa*c.CostoOrario) as BudgetEffettivo, sum(c.QuantitydiOutput*c.TempoRisorsa*b.CostoOrario)-sum(c.QuantitydiOutput*c.TempoRisorsa*c.CostoOrario) as scostamento
FROM impiegoConsuntivo as c join impiegoBudget as b 
on c.NrArticolo=b.NrArticolo and c.Risorsa=b.Risorsa and c.NrAreaProduzione=b.NrAreaProduzione
''')

scostamentoImpiegoPrezzo

,ImpiegoEffettivo,BudgetEffettivo,scostamento
0,5.341534e+06,5.341534e+06,0.0


In [121]:
scostamentoImpiegoVolume.iloc[0,2]+scostamentoImpiegoImpiego.iloc[0,2]+scostamentoImpiegoPrezzo.iloc[0,2]

958726.5971999932

# 3) scostamenti vendite (totale)

In [122]:


dfVendite.rename(columns = {'budget/cons':'budget'}, inplace = True)
dfVendite.loc[dfVendite.loc[:, "budget"] ==
              "Consuntivo", "budget"] = "CONSUNTIVO"
dfVendite.loc[dfVendite.loc[:, "budget"]
              == "Budget", "budget"] = "BUDGET"


#Joining with the exchange rate 
dfVendite = sqldf("SELECT * FROM  dfClienti C JOIN dfVendite V ON C.Nr = V.NrOrigine")

#La prima colonna è il codice del cliente

dfVendite = sqldf("SELECT * FROM dfVendite V JOIN dfCambio C ON  V.Valuta = C.CodiceValuta AND V.budget = C.Anno ")
del dfVendite['Anno']
del dfVendite['CodiceValuta']
del dfVendite['NrOrigine']

dfVendite

,Nr,Valuta,NrMovimento,budget,NrArticolo,Quantity,ImportoVenditaValutaLocaleTOTALEVENDITA,TassoCambioMedio
0,C00001,1,24907,BUDGET,ART0000893,2,250.0,1
1,C00001,1,31922,BUDGET,ART0005136,1,250.0,1
2,C00001,1,34528,BUDGET,ART0000877,4,952.0,1
3,C00001,1,34529,BUDGET,ART0000878,4,912.0,1
4,C00001,1,34530,BUDGET,ART0000879,4,312.0,1
...,...,...,...,...,...,...,...,...
2971,C00498,3,56896,CONSUNTIVO,ART0005136,16,0.0,"135,01"
2972,C00498,3,59224,CONSUNTIVO,ART0005136,16,204.0,"135,01"
2973,C00498,3,59225,CONSUNTIVO,ART0005136,16,122.4,"135,01"
2974,C00499,1,55977,CONSUNTIVO,ART0005136,1,250.0,1


## Splitting budget e consuntivo
### Budget

In [123]:
VenditeBudget = sqldf("SELECT * FROM dfVendite WHERE budget = 'BUDGET' or budget = 'Budget'")

VenditeBudget.head()


,Nr,Valuta,NrMovimento,budget,NrArticolo,Quantity,ImportoVenditaValutaLocaleTOTALEVENDITA,TassoCambioMedio
0,C00001,1,24907,BUDGET,ART0000893,2,250.0,1
1,C00001,1,31922,BUDGET,ART0005136,1,250.0,1
2,C00001,1,34528,BUDGET,ART0000877,4,952.0,1
3,C00001,1,34529,BUDGET,ART0000878,4,912.0,1
4,C00001,1,34530,BUDGET,ART0000879,4,312.0,1


### Consuntivo

In [124]:
VenditeConsuntivo = sqldf("SELECT * FROM dfVendite WHERE budget = 'CONSUNTIVO' or budget = 'Consuntivo'")

VenditeConsuntivo.head()


,Nr,Valuta,NrMovimento,budget,NrArticolo,Quantity,ImportoVenditaValutaLocaleTOTALEVENDITA,TassoCambioMedio
0,C00001,1,41582,CONSUNTIVO,ART0005136,1,175.0,1
1,C00001,1,44987,CONSUNTIVO,ART0000877,3,714.0,1
2,C00001,1,44988,CONSUNTIVO,ART0000878,3,684.0,1
3,C00001,1,44989,CONSUNTIVO,ART0000879,3,234.0,1
4,C00001,1,44990,CONSUNTIVO,ART0000880,12,48.0,1


### Calcolo dello scostamento

In [125]:
#Calcolo per il budget

#TODO-> AGGIUNGERE CONDIZIONE NELLA SELECT
#calcoloBudget = sqldf("SELECT * SUM(ImportoVenditaValutaLocaleTOTALEVENDITA*TassoCambioMedio) FROM VenditeBudget GROUP BY Nr")

#Calcolo per il consuntivo
#calcoloBudget = sqldf("SELECT * SUM(ImportoVenditaValutaLocaleTOTALEVENDITA*TassoCambioMedio) FROM VenditeConsuntivo GROUP BY Nr")


## scostamenti vendinte per NrArticolo

In [126]:
# quantità totale di codici MP 
countVenditec = sqldf('select distinct NrArticolo, count(*) from venditeConsuntivo group by NrArticolo order by count(*) desc')
countVenditeMPb = sqldf('select distinct NrArticolo, count(*) from venditeBudget group by NrArticolo order by count(*) desc')
print("Consuntivo")
print(countVenditec)
print("Budget")
print(countVenditeMPb)


PandaSQLException: (sqlite3.OperationalError) no such table: venditeConsuntivo
[SQL: select distinct NrArticolo, count(*) from venditeConsuntivo group by NrArticolo order by count(*) desc]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
venditeConsuntivoByMP = sqldf('select NrArticolo, sum(ImportoVenditaValutaLocaleTOTALEVENDITA) as sumVenditaTotale from venditeConsuntivo group by NrArticolo order by sumVenditaTotale desc')
venditeBudgetByMP = sqldf('select NrArticolo, sum(ImportoVenditaValutaLocaleTOTALEVENDITA) as sumVenditaTotale from venditeBudget group by NrArticolo order by sumVenditaTotale desc')


In [ ]:
scostamentoVenditeArt = sqldf('''select c.NrArticolo, b.sumVenditaTotale as sumVenditaTotaleBudget, c.sumVenditaTotale as sumVenditaTotaleConsuntivo, b.sumVenditaTotale-c.sumVenditaTotale as scostamento 
from venditeConsuntivoByMP as c join venditeBudgetByMP as b 
on c.NrArticolo = b.NrArticolo
order by scostamento desc''')

scostamentoVenditeArt

,NrArticolo,sumVenditaTotaleBudget,sumVenditaTotaleConsuntivo,scostamento
0,ART0000762,35334.60,7160.00,28174.60
1,ART0003780,6028.80,1312.00,4716.80
2,ART0000812,6429.60,2280.00,4149.60
3,ART0003610,4560.00,600.00,3960.00
4,ART0000826,4371.00,930.00,3441.00
...,...,...,...,...
408,ART0000811,7426.00,22120.00,-14694.00
409,ART0004889,450.00,16200.00,-15750.00
410,ART0005136,17920.21,34340.42,-16420.21
411,ART0000847,1776.38,21330.00,-19553.62


In [ ]:
sqldf('select sum (scostamento) from scostamentoVenditeArt') 
# notiamo che lo scostamento totale è completamente in linea con quello orginale :) <3


,sum (scostamento)
0,-198274.52


## scostamento di impiego e prezzo per NrArticolo

# Exporting results to create diagrams on Flutter

## New directory

In [ ]:
username = getpass.getuser()

print("Username: ", username)
pathPart1 = "/Users/"
pathPart2New = "/Github/SCGProject/Datasets/CsvForGraphing"
complePath = pathPart1+username+pathPart2New
dload = os.chdir(complePath)

if platform.system() == "Darwin":
    print("The OS is MACOS")
    if(username == "marcovinciguerra"):
        print("Complete path: ", complePath)
    elif(username == "davidguzman"):
        pathPart1 = "/Users/"
        pathPart2 = "/documents/Github/SCGProject/Datasets/CsvForGraphing"
        complePath = pathPart1+username+pathPart2
        dload = os.chdir(complePath)
elif platform.system() == "Linux":
    print("The OS is Linux")
    print("Complete path: ", complePath)

else:
    pathwin = r"C:\SCGProject\Datasets\CsvForGraphing"
    print("The OS is Windows")
    print("Complete path: ", pathwin)

    dload = os.chdir(pathwin)


Username:  marcovinciguerra
The OS is MACOS
Complete path:  /Users/marcovinciguerra/Github/SCGProject/Datasets/CsvForGraphing


## Saving data

In [ ]:
#1-> scostamentoConsumiMP
scostamentoConsumiArtImpiego.to_csv('scostamentoConsumiMP.csv')

#2-> scostamentoConsumiMP
scostamentoConsumiMP.to_csv('scostamentoConsumiMP.csv')

#3-> costamentoConsumiArt
scostamentoConsumiArt.to_csv('scostamentoConsumiArt.csv')